## Analyzing Stack Overflow BigQuery Data

## Author
* Matthias Jiro Walther
* https://www.linkedin.com/in/jirowalther/

---

# Central Questions
1. Which tags have the highest scoring questions on Stack Overflow?

2. Are users commenting on their own posts or others' posts more?

3. What tags on posts are attracting the most comments?

---


# Dataset Analysis

## Detailed Overview



I'm using three Stack Overflow data tables:
- bigquery-public-data.stackoverflow.comments
-- 16.03 GB and 86,754,111 rows


- bigquery-public-data.stackoverflow.stackoverflow_posts
-- 29.36 GB and 31,017,889 rows

- bigquery-public-data.stackoverflow.users
-- 3.14 GB and 18,712,212 rows

In [ ]:
from google.colab import auth
auth.authenticate_user()
project_id = "cs-145-project-1-474507"

In [ ]:
# Initialize BiqQuery client
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

## Investigation

In [ ]:
%%bigquery --project $project_id

# Title: Finding the most recent comment from each user

SELECT c.user_id, c.post_id, c.creation_date, c.score
FROM `bigquery-public-data.stackoverflow.comments` c
# Match the creation date with the latest date found in subquery
WHERE c.creation_date = (
  # subquery to find the latest comment date
  SELECT MAX(c2.creation_date)
  FROM `bigquery-public-data.stackoverflow.comments` c2
  WHERE c2.user_id = c.user_id
)
LIMIT 10;


In [ ]:
%%bigquery --project $project_id

# Title: Finding the most recent POST of each user

SELECT p.id, p.owner_user_id, p.title, p.creation_date
FROM `bigquery-public-data.stackoverflow.stackoverflow_posts` p
# Match the creation date with the latest post date found in subquery
WHERE p.creation_date = (
  # Subquery to find latest post date
  SELECT MAX(p2.creation_date)
  FROM `bigquery-public-data.stackoverflow.stackoverflow_posts` p2
  WHERE p2.owner_user_id = p.owner_user_id
)
LIMIT 10;

In [ ]:
%%bigquery --project $project_id

# Title: Finding the highest scoring post of users with over 100 up-votes

# CTE to find users who have over 100 up votes
WITH users_over_100_votes AS (
  SELECT id, reputation, up_votes, down_votes
  FROM `bigquery-public-data.stackoverflow.users`
  WHERE up_votes > 100
)

# Find the max post score for all users from CTE (limit 10)
SELECT p.owner_user_id, MAX(p.score) as max_score
FROM `bigquery-public-data.stackoverflow.stackoverflow_posts` p
JOIN users_over_100_votes u  # Join posts table to find posts made by users with over 100 up votes
ON p.owner_user_id = u.id
GROUP BY p.owner_user_id
LIMIT 10;

In [ ]:
%%bigquery --project $project_id

# Title: Top 10 users with reputation over 1000 whose posts have the highest average scores

# Find users with reputation score over 1000
WITH reputable_users AS (
  SELECT id
  FROM `bigquery-public-data.stackoverflow.users`
  WHERE reputation > 1000
)

# Find users average score among those in reputable_users
SELECT p.owner_user_id, AVG(p.score) as avg_score
FROM `bigquery-public-data.stackoverflow.stackoverflow_posts` p
JOIN reputable_users r
ON p.owner_user_id = r.id
GROUP BY p.owner_user_id
# Show the highest average scores
ORDER BY avg_score DESC
LIMIT 10;

In [ ]:
%%bigquery --project $project_id

# Title: Finding the top 10 users with highest reputation ranks

SELECT id, display_name, reputation, RANK() OVER (ORDER BY reputation DESC) AS reputation_rank
FROM `bigquery-public-data.stackoverflow.users`
ORDER BY reputation DESC
LIMIT 10;

In [ ]:
%%bigquery --project $project_id

# Title: Top 10 highest-scoring posts among users top-scoring posts

SELECT owner_user_id, id, score
FROM  (
  # Inner query groups all posts by owner_user_id, orders their posts by score
  SELECT owner_user_id, id, score,
  # Assign rank to each post among a specific user
  RANK() OVER (PARTITION BY owner_user_id ORDER BY score DESC) AS score_rank
  FROM `bigquery-public-data.stackoverflow.stackoverflow_posts`
  WHERE owner_user_id IS NOT NULL AND owner_user_id >= 0    # don't include stackoverflow general community: owner_user_id = -1
)
# Get top scoring post
WHERE score_rank = 1
ORDER BY score DESC
LIMIT 10;


In [ ]:
%%bigquery --project $project_id

# Title: Posts with scores higher than 100 with the lowest answer counts

SELECT id, owner_user_id, answer_count, body, score,
ROW_NUMBER() OVER (ORDER BY answer_count ASC) as answer_rank
FROM `bigquery-public-data.stackoverflow.stackoverflow_posts`
WHERE score > 100 and answer_count IS NOT NULL
ORDER BY answer_rank ASC
LIMIT 10;

# Exploring Central Questions



## Question 1: Which tags have the highest scoring questions on Stack Overflow?

In [ ]:
%%bigquery --project $project_id

# Find top 10 individual tags on Stack Overflow whose questions have the highest average score.
SELECT tag, AVG(p.score) AS avg_question_score
FROM `bigquery-public-data.stackoverflow.stackoverflow_posts` p
CROSS JOIN UNNEST(SPLIT(p.tags, '|')) AS tag        # Split tag strings into separate tags
WHERE p.tags IS NOT NULL AND p.post_type_id = 1     # No NULL and only from questions, not answers
GROUP BY tag
ORDER BY avg_question_score DESC                    # Order from highest to lowest average score
LIMIT 10;


This query shows us the top 10 individual tags whose associated questions have the highest average scores, showing us which topics on Stack Overflow are being positively received by the community. The top three are 'identity-operator', 'multiline-comments', and 'hints-and-tips'. These seem to be relatively simple tags, possibly being associated with beginner-level coding questions. Because these topics are accessible and helpful to new programmers, they may be more positively engaged with and thus have higher question scores.

In [ ]:
%%bigquery --project $project_id

# Finding the top users whose questions get the most positive reaction on average
# Score is based on average post score + comment score

WITH questions AS (
  # Fetch questions that have a known user who asked them
  SELECT id AS question_id, owner_user_id, score AS question_score
  FROM `bigquery-public-data.stackoverflow.stackoverflow_posts`
  WHERE post_type_id = 1 AND owner_user_id IS NOT NULL
),
comment_totals AS (
  # Fetch the sums of comment scores per post
  SELECT post_id AS question_id, SUM(score) AS comment_score_sum
  FROM `bigquery-public-data.stackoverflow.comments`
  GROUP BY post_id
),
joined AS (
  # Join questions to comment totals
  SELECT q.owner_user_id, q.question_id, q.question_score,
    IFNULL(c.comment_score_sum, 0) AS comment_score_sum   # Questions with no comments get a 0 (A.I. helped with this command)
  FROM questions q
  LEFT JOIN comment_totals c
  ON q.question_id = c.question_id
)

# For each asker of a question, compute average scores per question
SELECT owner_user_id, COUNT(*) AS n_questions, AVG(question_score) AS avg_question_score,
  AVG(comment_score_sum) AS avg_comment_score, AVG(question_score + comment_score_sum) AS avg_combined_score
FROM joined
GROUP BY owner_user_id
HAVING n_questions > 20
# Order users by their average combined score (question score average plus comment score average)
ORDER BY avg_combined_score DESC
LIMIT 10;

The results in this query show us the top ten Stack Overflow users whose questions are generating the most engagement, combining both the scores of their questions and their comments. We can see that these contributors are not only asking popular questions, but also facilitating conversation and exchanging knowledge with others.

## Question 2: Are users commenting on their own posts or others' posts more on Stack Overflow?

In [ ]:
%%bigquery --project $project_id

# Finding how many comments are of users commenting on their own posts versus others.
SELECT
  # Identify when user id in comment matches user id of post
  CASE WHEN c.user_id = p.owner_user_id THEN 'self_comment'
  ELSE 'other_comment' END AS comment_type,
  COUNT(*) num_comments             # Find total number of comments that are self comments or comments on others
FROM `bigquery-public-data.stackoverflow.comments` c
JOIN `bigquery-public-data.stackoverflow.stackoverflow_posts` p
ON c.post_id = p.id                 # Joining based on post id
WHERE c.user_id IS NOT NULL AND p.owner_user_id IS NOT NULL
GROUP BY comment_type;

# other_comment refers to users' comments on others posts
# self_comment refers to users' comments on own posts

We can see that of all the Stack Overflow comments, there were more than double the amount of comments on other users' posts versus commenting on one's own post. Users tend to comment on other users posts to answer their questions.

In [ ]:
%%bigquery --project $project_id

# Counting self comments and comments on others posts, separated by posts that are questions vs. answers
SELECT
  CASE WHEN c.user_id = p.owner_user_id THEN 'self_comment'
  ELSE 'other_comment' END AS comment_type,
  p.post_type_id,
  COUNT(*) num_comments     # Find total number of comments that are self comments or comments on others
FROM `bigquery-public-data.stackoverflow.comments` c
JOIN `bigquery-public-data.stackoverflow.stackoverflow_posts` p
ON c.post_id = p.id         # Joining based on post id
WHERE c.user_id IS NOT NULL AND p.owner_user_id IS NOT NULL AND p.post_type_id IN (1, 2)
# Group by the type of comment and the type of post (i.e. self comment vs. other comment, question vs. answer)
GROUP BY comment_type, p.post_type_id
ORDER BY p.post_type_id, comment_type;

After differentiating between post types, specifically posts that are questions or answers, we can see that users are more likely to comment on other users' posts rather than their own. However, we can see that self-commenting isn't rare. In fact, we can see that around 1 in 3 comments is from the original poster, suggesting that oftentimes there are follow-ups or responses to feedback from others.

## Question 3: What tags on Stack Overflow posts are attracting the most comments?

In [ ]:
%%bigquery --project $project_id

# Versus using comment_count in posts table, this query counts what's in the
# comments table which is slower but more precise

# CTE to count how many comments each post has
WITH per_post_comments AS (
  SELECT post_id, COUNT(*) as num_comments
  FROM `bigquery-public-data.stackoverflow.comments`
  GROUP BY post_id
)

# Find which tag combintations have the most total comments
SELECT p.tags, SUM(pc.num_comments) AS total_comments, COUNT(*) AS num_questions
FROM `bigquery-public-data.stackoverflow.stackoverflow_posts` p
JOIN per_post_comments pc          # Join CTE using post id's to get comment counts
ON p.id = pc.post_id
WHERE p.post_type_id = 1 AND p.tags IS NOT NULL
GROUP BY p.tags
ORDER BY total_comments DESC
LIMIT 10;

Based on this query, we can see that the posts with the tag 'java' in the question have the most comments. Following 'java', the tags 'php' and 'javascript' are the second and third most commented on posts.

In [ ]:
%%bigquery --project $project_id

# Use the comment_count column to identify tags in posts that garner the most comments
SELECT p.tags, SUM(p.comment_count) AS total_comments
FROM `bigquery-public-data.stackoverflow.stackoverflow_posts` p
WHERE p.tags IS NOT NULL and p.post_type_id = 1
GROUP BY p.tags
# Order by the total comments under posts with a specific tag, showing the highest first
ORDER BY total_comments DESC
LIMIT 10;

This query aims to answer the same question as the previous query, however, it does not utilize the comment table and instead uses the total from the posts table. I believe that the previous query will result in more accurate data as it is actually counting every comment in the comments table. It's reassuring to see that the top 10 tags remain the same across both queries.

# Findings


**Top Tags by Average Question Score:**

* The top 10 tags with the highest average question scores reveal which Stack Overflow topics are most positively received.

* The top three tags are ‘identity-operator,’ ‘multiline-comments,’ and ‘hints-and-tips.’

* These tags are likely tied to beginner-level or accessible topics, which may explain their higher engagement and positive reception.

**Top Users by Engagement:**
* The top 10 users were identified based on the combined scores of their questions and comments.

* These users not only ask popular questions but also drive conversations and knowledge sharing within the community.

**Commenting Behavior:**

* There are more than twice as many comments on other users’ posts compared to comments on one’s own posts.

* Users typically comment to help answer questions or contribute to discussions on others’ posts.

* However, self-commenting is fairly common — about 1 in 3 comments is made by the original poster.

* This suggests that users often follow up, clarify, or respond to feedback on their own posts.

**Query Comparison and Data Accuracy:**

* A secondary query was run using the posts table instead of the comments table to measure engagement.

* The previous query (using the comments table) is considered more accurate, as it counts every individual comment.

* It’s reassuring that the top 10 tags remained consistent across both methods, reinforcing the reliability of the findings.